In [155]:
import pandas as pd

df = pd.read_csv('usda_abbrev.csv')
myTable = pd.read_csv('Abbreviations_in_Shrt_Desc.csv')

In [156]:

# tokenize the items in the shrt_desc
import re

def tokenize_regex(text):
    RE_TOK = re.compile(r'([",&()-/]|\s)')    # tokenize with , & ( ) - /

    prev_idx = 0
    tokens = []
    for m in RE_TOK.finditer(text):
        t = text[prev_idx:m.start()].strip()
        if t: tokens.append(t)
        t = m.group().strip()
        if t:
            if tokens and tokens[-1] in {'W', 'WO'} and t == '/':   # tokenize W/ and WO/ as one token
                tokens[-1] = tokens[-1] + t
            else:
                tokens.append(t)
        prev_idx = m.end()

    t = text[prev_idx:].strip()
    if t: tokens.append(t)
    return tokens


In [157]:
count = 0
for desc in df['Shrt_Desc']:  # for each desc in Shrt_Desc column
    tokens = tokenize_regex(desc)   # tokenize the desc
    result = ""
    for token in tokens:   # for each token in tokens
        if token == "LESS" or token == "OIL" or token == "BLUE": result = result + " " + token      # these tokens are part of substrings in ABBREV, but we do not want to change them
        elif token == ',' or ")" or "(" or "+" or result[-1:] == "(": result = result + token
        elif token == ".": continue
        elif myTable['ABBREV'].str.fullmatch(token).any():      # if ABBREV contains the token
            token = myTable[myTable['ABBREV'].str.fullmatch(token)].iloc[0]['WORD']
            if (result[-1:] == "("): result = result + token.upper()
            else: result = result + " " + token.upper()    
        else: result = result + " " + token

    if result[0] == " ": result = result[1:]
    df.at[count,'Shrt_Desc'] = result
    count = count + 1

df.to_csv ("cleaned_usda.csv", index = False)
#print ("done")